In [3]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

# Class for the simulation

In [4]:
class Quantum_trajectory():
    def __init__(self, H, psi0, tlist, c_ops, ntraj=1):
        self.H = H
        self.psi0 = psi0
        self.tlist = tlist
        self.c_ops = c_ops
        self.ntraj = ntraj
        self.dt = tlist[1] - tlist[0]
        self.index = 0
        self.index_max = len(tlist)
        self.N_c_ops = len(c_ops)

    def decompose_rho(self, rho):

        # any density matrix can be decomposed into a sum of pure states, which is not unique as stated by the HJW therom, but we just need a single decomposition, so we chose the eigen states of the density matrix
        eigen_values, eigen_states = rho.eigenstates()
        return eigen_values, eigen_states

    def normelize_psi(self, psi):
        for i in range(len(psi)):
            psi[i] = psi[i] / np.sqrt(psi[i].dag() * psi[i])
        return psi

    def simulate_trajectory(self, psi):
        index = 0
        psi = []
        while index < self.index_max:
            R = np.random.rand()
            while psi[index].dag() * psi[index] > R and index < self.index_max:
                psi.append(self.time_evolve_psi(psi[index], index))
                index += 1
                if index >= self.index_max:
                    break
            else:
                P_jump = 0
                for i in range(self.N_c_ops):
                    P_jump += psi[index].dag() * self.c_ops[i].dag() * self.c_ops[i] * psi[index]
                Prob_jump = []
                for i in range(self.N_c_ops):
                    Prob_jump.append(psi[index].dag() * self.c_ops[i].dag() * self.c_ops[i] * psi[index] / P_jump)
                
                cumulative_prob = np.cumsum(Prob_jump)
                for i in cumulative_prob:
                    if R < i:
                        psi[index] = self.c_ops[i] * psi[index]
                        break
            psi[index] = psi[index] / np.sqrt(psi[index].dag() * psi[index])

        psi = self.normelize_psi(psi)

        return psi


    def choose_psi(self, eigen_values, eigen_states):
        R = np.random.rand()
        cumulative_prob = np.cumsum(eigen_values)
        for i in cumulative_prob:
            if R < i:
                return eigen_states[i]



    def time_evolve_psi(self, psi, index):

        d_psi_op = -1j * self.H[index]
        for i in range(len(self.c_ops)):
            d_psi_op += -self.c_ops[i].dag() * self.c_ops[i] / 2
        d_psi = d_psi_op * psi * self.dt
        psi =+ d_psi
        return psi


    def simulate(self, rho_0):
        # decompose the initial state into a sum of pure state wavefunctions
        eigen_values, eigen_states = rho_0.eigenstates()
        psi = []
        # run simulation for each trajectory
        for i in range(self.ntraj):
            # choose which pure state to simulate
            psi_0 = self.choose_psi(eigen_values, eigen_states)
            # simulate the trajectory
            psi.append(self.simulate_trajectory(psi_0))
        
        # aproximate the density matrix by averaging over all the pure states



        


SyntaxError: incomplete input (1429642433.py, line 1)

In [16]:
def decompose_rho(rho):

    # any density matrix can be decomposed into a sum of pure states, which is not unique as stated by the HJW therom, but we just need a single decomposition, so we chose the eigen states of the density matrix
    eigen_values, eigen_states = rho.eigenstates()
    
    rho = 0
    for i in range(len(eigen_states)):
        rho += eigen_values[i] * eigen_states[i] * eigen_states[i].dag()
    return rho

rho = qt.rand_dm(2)

print(type(rho))
rho_new = decompose_rho(rho)



print(rho - rho_new)

<class 'qutip.core.qobj.Qobj'>
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-2.22044605e-16  0.00000000e+00]
 [ 5.55111512e-17  3.88578059e-16]]
